# Retrieval Augmented Generation (RAG)
<sup>This notebook is a part of Natural Language Processing class at the University of Ljubljana, Faculty for computer and information science. Please contact [boshko.koloski@ijs.si](mailto:boshko.koloski@ijs.si) for any comments.</sub>




The core functionality of this notebook is to create a retrieval-augmented generation (RAG) system that enables discussion about the NLP subject setting using the `Mistral-7b-v0.2` model.

General-purpose language models can be fine-tuned for common tasks such as sentiment analysis and named entity recognition, which typically do not require additional background knowledge.

For more complex and knowledge-intensive tasks, it is possible to construct a language model-based system that accesses external knowledge sources. This approach enhances factual consistency, improves the reliability of responses, and mitigates the issue of 'hallucination.'

Researchers have introduced the Retrieval Augmented Generation (RAG) method to address these knowledge-intensive tasks. RAG integrates an information retrieval component with a text generator model, allowing for efficient updates and modifications to its internal knowledge without retraining the entire model.

RAG operates by taking an input, retrieving a set of relevant or supporting documents from a source like Wikipedia, and concatenating these documents with the original input prompt. This concatenated context is then fed to the text generator, which produces the final output. This adaptability is crucial for situations where facts may evolve over time, as the static parametric knowledge of traditional large language models (LLMs) can become outdated. RAG circumvents the need for retraining, providing access to the most current information and enabling reliable outputs through retrieval-based generation.

RAG requires additional document embeddings and the storage of documents in a database for retrieval purposes.


### Setup

Run the cells below to setup and install the required libraries. For our experiment we will need `bitsandbytes`, `accelerate`, `transformers`, `datasets`, `sentence-transformers` and  `faiss-gpu`.

In [1]:
!pip install langchain bitsandbytes accelerate sentence-transformers faiss-cpu langchain_community unstructured python-docx


[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
!pip install torch --index-url https://download.pytorch.org/whl/cu121



Looking in indexes: https://download.pytorch.org/whl/cu121
  Using cached https://download.pytorch.org/whl/cu121/torch-2.5.1%2Bcu121-cp311-cp311-win_amd64.whl (2449.4 MB)



[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())           # should be True
print(torch.version.cuda)                  # should show 12.1
print(torch.cuda.get_device_name(0))       # should return your GPU name


2.5.1+cu121
False
12.1


RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

**In simple terms, RAG is to LLMs what an open-book exam is to humans.**

The concept of an open-book exam centers around assessing a student's reasoning abilities rather than their capacity to memorize specific details. In a similar vein, RAG separates factual knowledge from the LLM’s reasoning capabilities. This factual information is stored in an external knowledge source, which is both easily accessible and updatable:

- **Parametric knowledge:** Knowledge that is learned during training and implicitly stored within the neural network's weights.
- **Non-parametric knowledge:** Information that is stored externally, for example, in a vector database.
e.

![RAG](RAG.jpg)

The RAG workflow consists of:

1. **The Retrieve**: The user query is used to retrieve relevant context from an external knowledge source. For this, the user query is embedded using an embedding model into the same vector space as the additional context in the vector database. This enables a similarity search, and the top k closest data objects from the vector database are returned.
2. **Augment**: The user query and the retrieved additional context are incorporated into a prompt template.
3. **Generate**: Finally, the retrieval-augmented prompt is fed to the LLM.

We will use the `langchain` framework to efficiently prompt the LLMs and prepare the RAG.


In [2]:
import os
import tempfile
from pathlib import Path
import joblib
import requests
import torch
import transformers

In [3]:
from IPython.display import display_markdown
from langchain.chains.conversational_retrieval.base import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter, HTMLHeaderTextSplitter, TokenTextSplitter
from langchain_community.document_loaders import BSHTMLLoader
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain_community.vectorstores.faiss import FAISS

In [4]:
LLM_MODEL = "mistralai/Mistral-7B-Instruct-v0.2"
device = f"cuda:{torch.cuda.current_device()}"

AssertionError: Torch not compiled with CUDA enabled

Large Language Models are known for their significant computational demands. Typically, the size of a model is determined by multiplying the number of parameters (size) by the precision of these values (data type). To conserve memory, weights can be stored using lower-precision data types through a process known as quantization.

**Post-Training Quantization (PTQ)** is a straightforward technique where the weights of an already trained model are converted to a lower precision without necessitating any retraining. Although easy to implement, PTQ can lead to potential performance degradation.We will employ PTQ using the `bitsandbytes` library and will load the model in 4-bit precision, applying double quantization with the `nf4` data type. For more information about quantization, visit [this guide on quantization](https://huggingface.co/docs/optimum/en/concept_guides/quantization) )pe.


In [15]:
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True, # loading in 4 bit
    bnb_4bit_quant_type="nf4", # quantization type
    bnb_4bit_use_double_quant=True, # nested quantization
    bnb_4bit_compute_dtype=torch.bfloat16,
)

In [22]:
!huggingface-cli login --token <INSERT_TOKEN>

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `jupyter-notebook` has been saved to C:\Users\sIGORs\.cache\huggingface\stored_tokens
Your token has been saved to C:\Users\sIGORs\.cache\huggingface\token
Login successful.
The current active token is: `jupyter-notebook`


In [25]:
model_config = transformers.AutoConfig.from_pretrained(
    pretrained_model_name_or_path=LLM_MODEL,
)
model = transformers.AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=LLM_MODEL,
    config=model_config,
    quantization_config=bnb_config, # we introduce the bnb config here.
    device_map="auto",
)
model.eval()

ImportError: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the numpy C-extensions failed. This error can happen for
many reasons, often due to issues with your setup or how NumPy was
installed.

We have compiled some common reasons and troubleshooting tips at:

    https://numpy.org/devdocs/user/troubleshooting-importerror.html

Please note and check the following:

  * The Python version is: Python3.11 from "C:\Users\sIGORs\AppData\Local\Programs\Python\Python311\python.exe"
  * The NumPy version is: "1.24.2"

and make sure that they are the versions you expect.
Please carefully study the documentation linked above for further help.

Original error was: DLL load failed while importing _multiarray_umath: The specified module could not be found.


RuntimeError: Failed to import transformers.models.auto.modeling_auto because of the following error (look up to see its traceback):
Failed to import transformers.generation.utils because of the following error (look up to see its traceback):
numpy._core.multiarray failed to import

We also need to load the tokenizer to transform the text as before.

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    pretrained_model_name_or_path=LLM_MODEL,
)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

We will use pipelines from Hugging Face to perform the prompting and generation with the Mistral model.


In [ ]:
generate_text = transformers.pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    #temperature=0.0,
    max_new_tokens=8192,
    repetition_penalty=1.1,
)

We will use `langchain` to link the HuggingFace models and the chaining prompting.

In [ ]:
llm = HuggingFacePipeline(pipeline=generate_text)

The core functionality of `langchain` is the creation of templates for prompting via `PromptTemplate`.

In [ ]:
from langchain_core.prompts import PromptTemplate

In [ ]:
template = """
You are a helpful AI QA assistant. When answering questions, use the context enclosed by triple backquotes if it is relevant.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Reply your answer in markdown format.
Question: {question}
Answer:"""
prompt = PromptTemplate.from_template(template)

In [ ]:
prompt

In [ ]:
chain = prompt | llm

In [ ]:
chain

In [ ]:
question = "What is the scoring criteria of the NLP course?"

In [ ]:
print(chain.invoke({"question": question}))

In [ ]:
question = "What do I have to do for the peer review?"

In [ ]:
print(chain.invoke({"question": question}))

**NOTE**: The model provides responses that are fairly general. We plan to enhance this by integrating contextual details from our course materials via **RAG**.

### Enter RAG

Next, we will define the vector embeddings of our context. We will use the `sentence-transformers/all-mpnet-base-v2` model to embed the documents and a FAISS vector store to store and later retrieve them. LangChain offers the `HuggingFaceEmbeddings` interface to easily load any model from Hugging Face to serve as the document representation.


In [ ]:
EMBED_MODEL = "sentence-transformers/all-mpnet-base-v2"

In [ ]:
embedding = HuggingFaceEmbeddings(
    model_name=EMBED_MODEL,
    model_kwargs={"device": "cuda"},
)

In [ ]:
def fetch_websites(sites: list[str]):
    docs = []
    with tempfile.TemporaryDirectory() as tmpdir:
        filename = f"{tmpdir}/site.html"
        for site in sites:
            res = requests.get(site)
            with open(filename, mode="wb") as fp:
                fp.write(res.content)
            docs.extend(BSHTMLLoader(filename, open_encoding = 'utf8').load())
    return docs

In [ ]:
course_webpage = "https://docs.google.com/document/d/e/2PACX-1vRwReAm5Vz_aiNyZN33eTz22fqMJhM0H-KtJdXthUw5cean_WYdBkZgJchP_s9th0rtUW0ikZZ_Fh5l/pub"

The `fetch_websites` function will be used to scrape data from our Google Document.


In [ ]:
docs = fetch_websites([course_webpage])

In [ ]:
docs[0]

We loaded the entire course into a single document. Since the sentence transformer can handle only limited sections of text, this might be problematic. Next, we will use the `RecursiveCharacterTextSplitter` to split the document into chunks with a `chunk_size` of 1000 characters and a `chunk_overlap` of 20.


In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
all_splits = text_splitter.split_documents(docs)

Let's see what the chunked document looks like.


In [ ]:
all_splits[0]

In [ ]:
all_splits[1]

In [ ]:
all_splits[2]

Next, we initialize a vector store. A vector store is a data structure that functions as a vector database, where each document is stored based on its own embedding. We will use the `FAISS` library for this purpose.


In [ ]:
vectorstore = FAISS.from_documents(all_splits, embedding)

Finally, we define a retriever—an object that will handle the **retrieving** part of the RAG pipeline. The retriever receives as arguments the metric by which we search the space and the number of the k-nearest documents (chunks in our case) that we retrieve to present to the studen


In [ ]:
retreiver = vectorstore.as_retriever(
    search_type="similarity",
    k=10,
)

Next, we modify our prompt template so that it now receives the context (the documents selected by the RAG system) and then generates the answer.


In [ ]:
PROMPT_TEMPLATE = """
You are a helpful AI QA assistant. When answering questions, use the context enclosed by triple backquotes if it is relevant.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Reply your answer in markdown format.

```
{context}
```

### Question:
{question}

### Answer:
"""

prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template=PROMPT_TEMPLATE.strip(),
)

With the prompt defined, we next set up the `ConversationalRetrievalChain` that will utilize the defined `retriever` and `llm`, following the `PROMPT_TEMPLATE` to extract documents.


In [ ]:
# Construct complete LLM chain
llm_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retreiver,
    return_source_documents=False,
    combine_docs_chain_kwargs={"prompt": prompt_template},
    verbose=False,
)

Finally, we create the `answer_question` function that will handle the chain invocation for us.


In [ ]:
def answer_question(question: str, history: dict[str] = None) -> str:
    if history is None:
        history = []

    response = llm_chain.invoke({"question": question, "chat_history": history})
    answer = response["answer"].split("### Answer:")[-1].strip()
    return answer

In [ ]:
question = "What do I have to do for the peer review?"
display_markdown(answer_question(question), raw=True)

In [ ]:
question = "What is the first project about?"
display_markdown(answer_question(question), raw=True)

In [ ]:
question = "What do you have to do for a good grade?"
display_markdown(answer_question(question), raw=True)

In [ ]:
question = "What is the project intended for digital linguistics about?"
display_markdown(answer_question(question), raw=True)

With the introduction of RAG, the `Mistral` model was able to successfully answer questions about the NLP course.

**Exercises:**
* Implement a data loader script that can load documents from a folder.
